# [LLM-Langchain應用 Arxiv論文機器人](https://kilong31442.medium.com/llm-langchain%E6%87%89%E7%94%A8-arxiv%E8%AB%96%E6%96%87%E6%A9%9F%E5%99%A8%E4%BA%BA-67c7298a9483)

In [ ]:
# from pprint import pprint
import pprint

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [ ]:
#載入必續的庫
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
#我們想要知道langsmith怎麼用所以載入langsmith的線上doc
loader=WebBaseLoader("https://docs.smith.langchain.com/")

docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever(search_kwargs={'k': 4})
retriever

用 "create_retriever_tool”建立檢索"工具"，之後要丟給Agent使用的。

這個funtion分別需要三個input:
* retriever(繼承自BaseRetriever)
* name:幫這個工具取個名字
* description: 幫這個工具的功能描述一下。

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(
    retriever=retriever,
    name="langsmith_search",
    description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [ ]:
retriever_tool.name

In [ ]:
retriever_tool.description

In [ ]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

In [ ]:
arxiv.description

In [ ]:
tools=[wiki, arxiv, retriever_tool]

In [ ]:
pprint.pprint(tools)

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0) #這邊使用gpt-3.5 turbo

In [ ]:
#使用hub 拉取 PromptTemplate工具
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

In [ ]:
### Agents
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})

In [ ]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})